<a href="https://www.kaggle.com/code/kevinuantela/crops-data-analysis?scriptVersionId=143969327" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import pearsonr

from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import GradientBoostingRegressor  # Para regresión
from sklearn import svm

very simple analysis to get a wide perspective of the data

In [ ]:
df = pd.read_csv("/kaggle/input/grains-and-cereals-futures/all_grains_data.csv")
print(df.head())
print(df.info())
print(df.describe())

In [ ]:
#transform data
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')

print(df.info())
print(df.head())

#NULLvalues
print(df.isnull().sum())

#clean data
uniqueVal=df['commodity'].unique()
print(uniqueVal)
uniqueVal=df['ticker'].unique()
print(uniqueVal)

#these values represent the same, one of them could be removed

In [ ]:
#basic data visualization to check all is correct and get a wide perspective of the data

#color palette
uniqueVal=df['commodity'].unique()
print(uniqueVal)

uniqueVal=df['ticker'].unique()
print(uniqueVal)

palette={'Corn':'gold','Oat':'orange','KC HRW Wheat':'magenta','Rough Rice':'green','Soybean Oil':'blue','Soybean':'black'}

plt.figure(1)
f1=plt.scatter(df['date'],df['open'],alpha=0.5, c=df['commodity'].map(palette),s=1)


The behaviour of each crop respect the open price is very similar but there is some small differences that could be interesting to note:
- In 2020 rice have peak, that not the others
- In 2000, until 2004, rice have a decrease while the others are stable
- Oat have a decrease in 2023 respect the others
- Soybean oil is very stable in the time
- Corn and wheat have a very similar behaviour but the peaks are more important in wheat
- Soybean, with a very similar behaviour than the others, have an increase in the time

In [ ]:
def graph(type):
	corn=df[df['commodity']==type]
	plt.plot(corn['date'],corn['open'],c='b',lw=0.5)
	plt.plot(corn['date'],corn['high'],c='g',lw=0.5)
	plt.plot(corn['date'],corn['low'],c='g',lw=0.5)
	plt.plot(corn['date'],corn['close'],c='r',lw=0.5)

plt.figure(5)

graph('KC HRW Wheat')
graph('Rough Rice')
graph('Corn')
graph('Oat')
graph('Soybean Oil')
graph('Soybean')

not important changes between open, close, low and higher

In [ ]:
#Evaluation of multicolinearity

#correlation matrix
correlation_matrix = df[['open', 'high','low','close']].corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')


very high correalations
check the values

In [ ]:
correlations = []
p_values = []
correlation, p_value = pearsonr(df['open'], df['high'])
correlations.append(correlation)
p_values.append(p_value)
correlation, p_value = pearsonr(df['open'], df['close'])
correlations.append(correlation)
p_values.append(p_value)
correlation, p_value = pearsonr(df['open'], df['low'])
correlations.append(correlation)
p_values.append(p_value)
correlation, p_value = pearsonr(df['high'],df['low'])
correlations.append(correlation)
p_values.append(p_value)
correlation, p_value = pearsonr(df['high'],df['close'])
correlations.append(correlation)
p_values.append(p_value)
correlation, p_value = pearsonr(df['low'], df['close'])
correlations.append(correlation)
p_values.append(p_value)

for i in range(0,6):
	print(f'{correlations[i]}')
	print(f'{p_values[i]:.10f}')


very high R^2 and p-values<0.05
these parameters provide the same information, problem of multilinearity, make the prediction models only with one of them

In [ ]:
#only for check the regression of two of these parameters
plt.figure(33)
plt.scatter(df['open'],df['close'],s=1)
regression = LinearRegression()
x=df['open']
x=x.values.reshape(-1, 1)
y=df['close']
regression.fit(x,y)
Y_pred = regression.predict(x)
plt.plot(x, Y_pred, label='Línea de regresión', color='red')
r2 = r2_score(y, Y_pred)
m = regression.coef_[0]
b = regression.intercept_
ecuacion_recta = f'Recta: y = {m:.2f}x + {b:.2f}'
r2_text = f'R^2 = {r2:.2f}'
plt.text(0.2, 0.9, ecuacion_recta, fontsize=12, transform=plt.gca().transAxes)
plt.text(0.2, 0.85, r2_text, fontsize=12, transform=plt.gca().transAxes)

it is possible to explore other parameter: variation between the high and the low price

In [ ]:
variance= np.array(df['high']-df['low'])
print(variance)
df['variance']=variance

def vargraph(type,fig):
	corn=df[df['commodity']==type]
	plt.figure(fig)
	plt.plot(corn['date'],corn['variance']/max(corn['variance']),c='b',lw=0.5)

vargraph('Corn',6)
vargraph('Oat',7)
vargraph('KC HRW Wheat',8)
vargraph('Rough Rice',9)
vargraph('Soybean Oil',10)
vargraph('Soybean',11)

It is possible to check that variations are different for each type of crop, this parameter could be interesting to add to the DF

In [ ]:
plt.figure(34)
plt.scatter(df['open'],df['variance'],s=1)
regression = LinearRegression()
x=df['open']
x=x.values.reshape(-1, 1)
y=df['variance']
regression.fit(x,y)
Y_pred = regression.predict(x)
plt.plot(x, Y_pred, label='Línea de regresión', color='red')
r2 = r2_score(y, Y_pred)
m = regression.coef_[0]
b = regression.intercept_
ecuacion_recta = f'Recta: y = {m:.2f}x + {b:.2f}'
r2_text = f'R^2 = {r2:.2f}'
plt.text(0.2, 0.9, ecuacion_recta, fontsize=12, transform=plt.gca().transAxes)
plt.text(0.2, 0.85, r2_text, fontsize=12, transform=plt.gca().transAxes)

Variance and price don´t show an important correlation, provide different information

In [ ]:
plt.figure(35)

correlation_matrix = df[['open', 'high','low','close','variance']].corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')



corn=df[df['commodity']=='Corn']
corn2=df[df['commodity']=='Oat']
corn3=df[df['commodity']=='KC HRW Wheat']
corn4=df[df['commodity']=='Rough Rice']
corn5=df[df['commodity']=='Soybean Oil']
corn6=df[df['commodity']=='Soybean']

merged_df = pd.merge(corn,corn2, on='date', how='inner',suffixes=('_corn1', '_corn2'))
merged_df = pd.merge(merged_df,corn3, on='date', how='inner',suffixes=('_merged', '_corn3'))
merged_df = pd.merge(merged_df,corn4, on='date', how='inner',suffixes=('_merged2', '_corn4'))
merged_df = pd.merge(merged_df,corn5, on='date', how='inner',suffixes=('_merged3', '_corn5'))
merged_df = pd.merge(merged_df,corn6, on='date', how='inner',suffixes=('_merged4', '_corn6'))

merged_df.columns=['ticker_corn','com_corn','date','op_corn','hi_corn','low_corn','clo_corn','vol_corn','var_corn',
'ticker_Oat','com_Oat','op_Oat','hi_Oat','low_Oat','clo_Oat','vol_Oat','var_Oat',
'ticker_KC HRW Wheat','com_KC HRW Wheat','op_KC HRW Wheat','hi_KC HRW Wheat','low_KC HRW Wheat','clo_KC HRW Wheat','vol_KC HRW Wheat','var_KC HRW Wheat',
'ticker_Rough Rice','com_Rough Rice','op_Rough Rice','hi_Rough Rice','low_Rough Rice','clo_Rough Rice','vol_Rough Rice','var_Rough Rice',
'ticker_Soybean Oil','com_Soybean Oil','op_Soybean Oil','hi_Soybean Oil','low_Soybean Oil','clo_Soybean Oil','vol_Soybean Oil','var_Soybean Oil',
'ticker_Soybean','com_Soybean','op_Soybean','hi_Soybean','low_Soybean','clo_Soybean','vol_Soybean','var_Soybean']

print(merged_df.info())

as could be possible to check before, the 4 parameters provide the same information but no the variance

In [ ]:
plt.figure(36)

correlation_matrix = merged_df[['op_corn','hi_corn','low_corn','clo_corn','vol_corn','var_corn',
'op_Oat','hi_Oat','low_Oat','clo_Oat','vol_Oat','var_Oat',
'op_KC HRW Wheat','hi_KC HRW Wheat','low_KC HRW Wheat','clo_KC HRW Wheat','vol_KC HRW Wheat','var_KC HRW Wheat',
'op_Rough Rice','hi_Rough Rice','low_Rough Rice','clo_Rough Rice','vol_Rough Rice','var_Rough Rice',
'op_Soybean Oil','hi_Soybean Oil','low_Soybean Oil','clo_Soybean Oil','vol_Soybean Oil','var_Soybean Oil',
'op_Soybean','hi_Soybean','low_Soybean','clo_Soybean','vol_Soybean','var_Soybean']].corr()

sns.heatmap(correlation_matrix, annot=False, cmap='coolwarm')

All type of studies show that it is neccessary to remove 3/4 parameters (high, low, open, close). Open is going to be kept due is the first price in the day and this could be more interesting 

In [ ]:
final_df = merged_df.drop(columns=['ticker_corn','hi_corn','low_corn','clo_corn',
'ticker_Oat','hi_Oat','low_Oat','clo_Oat',
'ticker_Rough Rice','hi_Rough Rice','low_Rough Rice','clo_Rough Rice',
'ticker_KC HRW Wheat','hi_KC HRW Wheat','low_KC HRW Wheat','clo_KC HRW Wheat',
'ticker_Soybean Oil','hi_Soybean Oil','low_Soybean Oil','clo_Soybean Oil',
'ticker_Soybean','hi_Soybean','low_Soybean','clo_Soybean'])

print(final_df.info())

plt.figure(37)

correlation_matrix = final_df[['op_corn','vol_corn','var_corn',
'op_Oat','vol_Oat','var_Oat',
'op_KC HRW Wheat','vol_KC HRW Wheat','var_KC HRW Wheat',
'op_Rough Rice','vol_Rough Rice','var_Rough Rice',
'op_Soybean Oil','vol_Soybean Oil','var_Soybean Oil',
'op_Soybean','vol_Soybean','var_Soybean']].corr()
print(correlation_matrix)

sns.heatmap(correlation_matrix, annot=False, cmap='coolwarm')


These parameters could provide a good database to make prediction models

In [ ]:
#data to work
work_df=df.drop(columns=['ticker','high','low','close'])
work_df['month']= work_df['date'].dt.month
work_df['year']= work_df['date'].dt.year
work_df['day']= work_df['date'].dt.day
print(work_df.info())

plt.figure(38)
correlation_matrix = work_df[['open','volume','variance','day','month','year']].corr()

sns.heatmap(correlation_matrix, annot=False, cmap='coolwarm')

In [ ]:
#First Challenge
#stratified split

stratified_splitter = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
X = work_df[['commodity', 'month', 'year','day', 'open', 'variance']]
y = work_df['volume']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=pd.qcut(y, q=4))

stats_train = y_train.describe()
stats_test = y_test.describe()
print(stats_train)
print(stats_test)

Split: correct

In [ ]:
#linear model

X = pd.get_dummies(X, columns=['commodity'], drop_first=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=pd.qcut(y, q=4))

stats_train = y_train.describe()
stats_test = y_test.describe()
print(stats_train)
print(stats_test)

modelo_regresion_multiple = LinearRegression()
modelo_regresion_multiple.fit(X_train, y_train)

y_pred = modelo_regresion_multiple.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Error Cuadrático Medio (MSE): {mse}')
print(f'Coeficiente de Determinación (R²): {r2}')

coeficientes = modelo_regresion_multiple.coef_
intercept = modelo_regresion_multiple.intercept_

print('Coeficientes:', coeficientes)
print('Intercept:', intercept)


R^2= 0.434
MSE= 1.550.000.000
not good model

In [ ]:
#random forest

#param_grid = {
#    'n_estimators': [100, 500],
#    'max_depth': [10,  30,  50, None],
#    'min_samples_split': [2, 5],
#    'min_samples_leaf': [ 2, 4],
#    'bootstrap': [True, False]
#}
#was previoously tested
#best parameters
param_grid = {
    'n_estimators': [500],
    'max_depth': [50],
    'min_samples_split': [5],
    'min_samples_leaf': [2],
    'bootstrap': [True]
}
rf_model = RandomForestRegressor(random_state=42)

random_search = RandomizedSearchCV(
    estimator=rf_model,
    param_distributions=param_grid,
    n_iter=1, # Número de combinaciones aleatorias a probar
    cv=5,         # Número de divisiones para la validación cruzada
    random_state=42,
    n_jobs=-1     # Utiliza todos los núcleos de la CPU para la búsqueda
)


random_search.fit(X_train, y_train)

best_params = random_search.best_params_
best_model = random_search.best_estimator_
#print("Mejores hiperparámetros:", best_params)

best_model.fit(X_train, y_train)

y_pred = best_model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

r2 = r2_score(y_test, y_pred)
print(f"R-squared: {r2}")
feature_importance = best_model.feature_importances_
feature_names = X.columns
plt.figure(figsize=(10, 6))
plt.barh(range(len(feature_names)), feature_importance, align='center')
plt.yticks(range(len(feature_names)), feature_names)
plt.xlabel('Importancia de las Características')
plt.title('Importancia de las Características en el Modelo de Random Forest')

plt.figure(40)
y_test2=y_test.values.reshape(-1, 1)
regression.fit(y_test2,y_pred)

plt.scatter(y_test2,y_pred,s=1)
r2 = r2_score(y_test2,y_pred)
m = regression.coef_[0]
b = regression.intercept_
ecuacion_recta = f'Recta: y = {m:.2f}x + {b:.2f}'
r2_text = f'R^2 = {r2:.2f}'
plt.text(0.2, 0.9, ecuacion_recta, fontsize=12, transform=plt.gca().transAxes)
plt.text(0.2, 0.85, r2_text, fontsize=12, transform=plt.gca().transAxes)

previously I tested the data without the day and R^2 was 0.786 and open had  a very high weight in the model.
This show that add 'day' to the model provide a very important information
R^2=0.92
MSE= 230.000.000
slope=0.90 (near to 1, but in this case x=0 have the highest errors and distorts the slope, could be a possible explanation)
intercept point = 2814 (1% of the higest value)
very good prediction model
A last model is gonna be tested

In [ ]:
#gradient boost

gb_model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, random_state=42)  # Ajusta los hiperparámetros según tus necesidades

gb_model.fit(X_train, y_train)

y_pred = gb_model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R-squared: {r2}")

This model provide a R^2= 0.640 and a MSE= 988.000.000
The best model of the tested models for yield crop prediction is random forest.
As yield crop indicator volume of contracts signed has been studied. This is due it is possible to understand that a higher yield of the crops provide a higher amount of contracts. Due the lack of data and infoormation about the data, this parameter has been taken

In [ ]:
#second challenge
#there is not weather data available for the analysis, i can use the 'date' parameter to understand the weather factor attending to seasons
#using 'date' parameter for this turn this challenge into the third challenge

In [ ]:
#third challenge
#in this case in order to know the price, the volume is a data due to the price, for this this parameter can´t be used, is the same for the variance
#in order to know the price, in a simple way, only we can use the type of crop and the 'date'stratified_splitter = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

X = work_df[['commodity', 'month', 'year','day']]
y = work_df['open']

X = pd.get_dummies(X, columns=['commodity'], drop_first=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=pd.qcut(y, q=4))

stats_train = y_train.describe()
stats_test = y_test.describe()
print(stats_train)
print(stats_test)

Split: correct

The same parameters of the previous model (to reduce the time) are gonna be used

In [ ]:
param_grid = {
    'n_estimators': [500],
    'max_depth': [50],
    'min_samples_split': [5],
    'min_samples_leaf': [2],
    'bootstrap': [True]
}


rf_model = RandomForestRegressor(random_state=42)

random_search = RandomizedSearchCV(
    estimator=rf_model,
    param_distributions=param_grid,
    n_iter=1, # Número de combinaciones aleatorias a probar
    cv=5,         # Número de divisiones para la validación cruzada
    random_state=42,
    n_jobs=-1     # Utiliza todos los núcleos de la CPU para la búsqueda
)


random_search.fit(X_train, y_train)

best_params = random_search.best_params_
best_model = random_search.best_estimator_
print("Mejores hiperparámetros:", best_params)

best_model.fit(X_train, y_train)

y_pred = best_model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

r2 = r2_score(y_test, y_pred)
print(f"R-squared: {r2}")

feature_importance = best_model.feature_importances_
feature_names = X.columns
plt.figure(42)
plt.figure(figsize=(10, 6))
plt.barh(range(len(feature_names)), feature_importance, align='center')
plt.yticks(range(len(feature_names)), feature_names)
plt.xlabel('Importancia de las Características')
plt.title('Importancia de las Características en el Modelo de Random Forest')

plt.figure(41)
y_test2=y_test.values.reshape(-1, 1)
regression.fit(y_test2,y_pred)

plt.scatter(y_test2,y_pred,s=1)
r2 = r2_score(y_test2,y_pred)
m = regression.coef_[0]
b = regression.intercept_
ecuacion_recta = f'Recta: y = {m:.2f}x + {b:.2f}'
r2_text = f'R^2 = {r2:.5f}'
plt.text(0.2, 0.9, ecuacion_recta, fontsize=12, transform=plt.gca().transAxes)
plt.text(0.2, 0.85, r2_text, fontsize=12, transform=plt.gca().transAxes)


In [ ]:
plt.scatter(y_test2,y_pred,s=1)
r2 = r2_score(y_test2,y_pred)
m = regression.coef_[0]
b = regression.intercept_
ecuacion_recta = f'Recta: y = {m:.2f}x + {b:.2f}'
r2_text = f'R^2 = {r2:.5f}'
plt.text(0.2, 0.9, ecuacion_recta, fontsize=12, transform=plt.gca().transAxes)
plt.text(0.2, 0.85, r2_text, fontsize=12, transform=plt.gca().transAxes)

With a slope of 1, intercept point of 0.26 and a R^2=0.9987 it is possible to say that random forest, bearing in mind the type of crop and the date, provides a very good model prediction for the open price, with this it is possible to predict the volume of contract signed with the other model.

The parameters more important to this model are the type of crap and the year, seems that month provides some infromation but the day is not very important. This is consistent with the firs graph.

In the first graph it is possible to see some stable parts in the time (in the same year) with little peaks (could be the month), and this is modified according the type of crop. for this is reasonable that the most important parameter (without take in count the type of crop) would be the year, and with the month understand the small modifications in the stable trend during the year.